Create a "debiased matrix" (on the axis of gender) in which the shapes of male and female distributions are the same (excluding appropriately gendered words) which represents the conditional probability of each word pair.

In [123]:
from gensim.models import Word2Vec
import logging
import math
import numpy as np
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim.downloader as api
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import ot
# necessary for 3d plot even if not used
from mpl_toolkits.mplot3d import Axes3D  # noqa
from matplotlib.collections import PolyCollection
import json
import time
from sklearn.linear_model import LinearRegression
import tensorflow as tf

In [124]:
wiki_model = Word2Vec.load("english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model")


2019-07-29 20:42:02,773 : INFO : loading Word2Vec object from english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model
2019-07-29 20:42:11,069 : INFO : loading wv recursively from english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.wv.* with mmap=None
2019-07-29 20:42:11,074 : INFO : loading vectors from english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.wv.vectors.npy with mmap=None
2019-07-29 20:42:11,213 : INFO : loading vectors_vocab from english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.wv.vectors_vocab.npy with mmap=None
2019-07-29 20:42:11,337 : INFO : loading vectors_ngrams from english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.wv.vectors_ngrams.npy with mmap=None
2019-07-29 20:42:11,751 : INFO : setting ignored attribute vectors_norm to None
2019-07-29 20:42:11,753 : INFO : setting ignored attribute vectors_vocab_norm to None
2019-07-29 20:42:11,763 : INFO : 

In [125]:
vocab_size = 22000
k = 20

Per Mikolov's paper "Distributed Representations of Words and Phrases and their Compositionality," we use the following definition of each log conditional probability.

$$ \log P(w_O|w_I) \approx \log \sigma ({v'_{wo}}^T v_{wI}) + \sum_{i=1}^{k} [\log {\sigma ({{-v'_{wi}}^T v_{wI}})}] $$

https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

To select the k samples, we similarly follow the methodology in the original paper and construct a unigram distribution raised to the power of (3/4). 

$$P(w_i) =\frac{f(w_i)^{3/4}}{\sum_{j=0}^{n} (f(w_j)^{3/4})}$$

In [126]:
word_counts = np.array([wiki_model.wv.vocab[wiki_model.wv.index2word[i]].count for i in range(vocab_size)])
word_counts_power = np.power(word_counts,.75)
sampling_dist = np.true_divide(word_counts_power,np.sum(word_counts_power))


Construct the probability matrix

In [127]:
def sigmoid(x):
    return 1 / (1 + np.e ** -x)

In [128]:
wvs = np.load('english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.wv.vectors.npy')
wvs = wvs[:vocab_size]


In [129]:
cvs = np.load('english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.trainables.syn1neg.npy')
cvs = cvs[:vocab_size]


In [130]:
cond_probs = np.load('cond_probs_alt.npy')

TODO: add more gender word pairs (girlfriend, boyfriend, etc)

In [131]:
gender_word_pairs = [('he','she'),('man','woman'),('his','her'),('himself','herself'), ('him','her'),('men','women'),('husband','wife'),('girl','boy'),('men','women'),('brother','sister'),('mother','father'),('aunt','uncle'),('grandfather','grandmother'),('son','daughter'),('waiter','waitress'),('niece','nephew')]
gender_word_pairs_simple = [w for p in gender_word_pairs for w in p] 
gw_dict = {w:i for (i,w) in enumerate(gender_word_pairs_simple)}


In [132]:
gw_dict

{'aunt': 22,
 'boy': 15,
 'brother': 18,
 'daughter': 27,
 'father': 21,
 'girl': 14,
 'grandfather': 24,
 'grandmother': 25,
 'he': 0,
 'her': 9,
 'herself': 7,
 'him': 8,
 'himself': 6,
 'his': 4,
 'husband': 12,
 'man': 2,
 'men': 16,
 'mother': 20,
 'nephew': 31,
 'niece': 30,
 'she': 1,
 'sister': 19,
 'son': 26,
 'uncle': 23,
 'waiter': 28,
 'waitress': 29,
 'wife': 13,
 'woman': 3,
 'women': 17}

Ensure no words have index greater than `vocab_size`

In [133]:
for gw in gender_word_pairs_simple:
    j = wiki_model.wv.vocab[gw].index
    if j > vocab_size:
        print('{} has index {}, too large for vocab size {}').format(gw,j,vocab_size)

In [138]:
cond_probs = np.full((vocab_size,vocab_size),-10.)
cond_probs.shape

(22000, 22000)

In [ ]:
start = time.time()
for i in range(vocab_size):
    for gw in gender_word_pairs_simple:
        gw_i = wiki_model.wv.vocab[gw].index
        v_prime_wo = wvs[i]
        v_wi = cvs[gw_i]
        samples = np.random.choice(vocab_size,size=k,replace=False,p=sampling_dist)
        first_term = np.log(sigmoid((np.matmul(v_prime_wo,v_wi))))
        second_term = np.sum([np.log(sigmoid(-1*np.matmul(wvs[s],v_wi))) for s in samples])
        cond_probs[i,gw_i] = np.exp(first_term + second_term)
end = time.time()
print(end - start)

In [76]:
# cond_probs = cond_probs / np.sum(cond_probs,axis=0)

In [77]:
np.save('cond_probs_alt',cond_probs)

In [78]:
appropriately_gendered_words = ["he", "his", "her", "she", "him", "man", "women", "men", "woman", "spokesman", "wife", "himself", "son", "mother", "father", "chairman",
"daughter", "husband", "guy", "girls", "girl", "boy", "boys", "brother", "spokeswoman", "female", "sister", "male", "herself", "brothers", "dad",
"actress", "mom", "sons", "girlfriend", "daughters", "lady", "boyfriend", "sisters", "mothers", "king", "businessman", "grandmother",
"grandfather", "deer", "ladies", "uncle", "males", "congressman", "grandson", "bull", "queen", "businessmen", "wives", "widow",
"nephew", "bride", "females", "aunt", "prostate cancer", "lesbian", "chairwoman", "fathers", "moms", "maiden", "granddaughter",
"younger brother", "lads", "lion", "gentleman", "fraternity", "bachelor", "niece", "bulls", "husbands", "prince", "colt", "salesman", "hers",
"dude", "beard", "filly", "princess", "lesbians", "councilman", "actresses", "gentlemen", "stepfather", "monks", "ex girlfriend", "lad",
"sperm", "testosterone", "nephews", "maid", "daddy", "mare", "fiance", "fiancee", "kings", "dads", "waitress", "maternal", "heroine",
"nieces", "girlfriends", "sir", "stud", "mistress", "lions", "estranged wife", "womb", "grandma", "maternity", "estrogen", "ex boyfriend",
"widows", "gelding", "diva", "teenage girls", "nuns", "czar", "ovarian cancer", "countrymen", "teenage girl", "penis", "bloke", "nun",
"brides", "housewife", "spokesmen", "suitors", "menopause", "monastery", "motherhood", "brethren", "stepmother", "prostate",
"hostess", "twin brother", "schoolboy", "brotherhood", "fillies", "stepson", "congresswoman", "uncles", "witch", "monk", "viagra",
"paternity", "suitor", "sorority", "macho", "businesswoman", "eldest son", "gal", "statesman", "schoolgirl", "fathered", "goddess",
"hubby", "stepdaughter", "blokes", "dudes", "strongman", "uterus", "grandsons", "studs", "mama", "godfather", "hens", "hen", "mommy",
"estranged husband", "elder brother", "boyhood", "baritone", "grandmothers", "grandpa", "boyfriends", "feminism", "countryman",
"stallion", "heiress", "queens", "witches", "aunts", "semen", "fella", "granddaughters", "chap", "widower", "salesmen", "convent",
"vagina", "beau", "beards", "handyman", "twin sister", "maids", "gals", "housewives", "horsemen", "obstetrics", "fatherhood",
"councilwoman", "princes", "matriarch", "colts", "ma", "fraternities", "pa", "fellas", "councilmen", "dowry", "barbershop", "fraternal",
"ballerina"]

In [79]:
def get_indices_in_matrix(words):
    def get_index(w):
        try: 
            return wiki_model.wv.vocab[w].index
        except:
            print("not in model: {}".format(w))
            return vocab_size+10
    indices = list(map(lambda word: get_index(word), words))
    indices = list(filter(lambda i: i < vocab_size, indices))
    return np.flip(np.sort(indices))

In [80]:
app_gen_indices = get_indices_in_matrix(appropriately_gendered_words)

not in model: prostate cancer
not in model: younger brother
not in model: ex girlfriend
not in model: estranged wife
not in model: ex boyfriend
not in model: teenage girls
not in model: ovarian cancer
not in model: teenage girl
not in model: twin brother
not in model: eldest son
not in model: hubby
not in model: estranged husband
not in model: elder brother
not in model: twin sister


In [81]:
inapp_gen_indices = np.array(list((filter(lambda x: x not in app_gen_indices, np.arange(vocab_size)))))
inapp_gen_indices

array([    0,     1,     2, ..., 21997, 21998, 21999])

In [58]:
def debias_vector_pairs(gender_pairs,model):
    new_mat = cond_probs.copy()
    for p1,p2 in gender_pairs:
        print('debiasing distributions {} and {}'.format(p1,p2))
        p1_index,p2_index = wiki_model.wv.vocab[p1].index,wiki_model.wv.vocab[p2].index
        p1_dist = new_mat[inapp_gen_indices,p1_index]
        p2_dist = new_mat[inapp_gen_indices,p2_index]
#         print(p1_index)
#         print(p2_index)
        normed_p1_dist,normed_p2_dist = p1_dist/np.sum(p1_dist), p2_dist/np.sum(p2_dist)
        A = np.vstack((normed_p1_dist, normed_p2_dist)).T
        n_distributions = A.shape[1]
        M = ot.utils.dist0(len(p1_dist))
        M /= M.max()
        alpha = 0.2  # 0<=alpha<=1
        weights = np.array([1 - alpha, alpha])

        # wasserstein
        reg = 1e2
        bary_wass = ot.bregman.barycenter(A, M, reg, weights)

        new_mat[inapp_gen_indices,p1_index] = bary_wass * np.sum(p1_dist)
        new_mat[inapp_gen_indices,p2_index] = bary_wass * np.sum(p2_dist)
    return new_mat

NOTE: `debias_vector_pairs` takes ~1 hour for `vocab_size` 22000

In [82]:
new_mat = debias_vector_pairs(gender_word_pairs,wiki_model)

debiasing distributions he and she
debiasing distributions man and woman
debiasing distributions his and her
debiasing distributions himself and herself
debiasing distributions him and her
debiasing distributions men and women
debiasing distributions husband and wife
debiasing distributions girl and boy
debiasing distributions men and women
debiasing distributions brother and sister
debiasing distributions mother and father
debiasing distributions aunt and uncle
debiasing distributions grandfather and grandmother
debiasing distributions son and daughter
debiasing distributions waiter and waitress
debiasing distributions niece and nephew


In [83]:
np.save('debiased_matrix_alt',new_mat)

In [62]:
# debiased_matrix_alt = np.load('debiased_matrix_alt.npy')

In [84]:
new_mat.sum(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
for gw in gender_word_pairs_simple:
    j = wiki_model.wv.vocab[gw].index

In [115]:
debiased_matrix_alt_orig = np.load('debiased_matrix_alt_orig.npy')

In [116]:
gender_word_pairs_simple[:-4]

['he',
 'she',
 'man',
 'woman',
 'his',
 'her',
 'himself',
 'herself',
 'him',
 'her',
 'men',
 'women',
 'husband',
 'wife',
 'girl',
 'boy',
 'men',
 'women',
 'brother',
 'sister',
 'mother',
 'father',
 'aunt',
 'uncle',
 'grandfather',
 'grandmother',
 'son',
 'daughter']

In [117]:
gender_inds = np.array([wiki_model.wv.vocab[w].index for w in gender_word_pairs_simple[:-4]])

In [118]:
debiased_matrix_alt_new = debiased_matrix_alt_orig[:,gender_inds]


In [119]:
debiased_matrix_alt_new

array([[0.00145599, 0.0009459 , 0.00078978, ..., 0.00764814, 0.00196016,
        0.00335457],
       [0.00145599, 0.0009459 , 0.00078978, ..., 0.00764814, 0.00196016,
        0.00335458],
       [0.00145599, 0.0009459 , 0.00078978, ..., 0.00764814, 0.00196016,
        0.00335458],
       ...,
       [0.00145503, 0.00094527, 0.00078967, ..., 0.0076529 , 0.0019618 ,
        0.00335737],
       [0.00145503, 0.00094527, 0.00078967, ..., 0.00765289, 0.00196179,
        0.00335737],
       [0.00145503, 0.00094527, 0.00078967, ..., 0.00765289, 0.00196179,
        0.00335737]])

In [120]:
debiased_matrix_alt_new.shape

(22000, 28)

In [121]:
debiased_matrix_alt_new.sum(axis=0)

array([ 3.20817376e+01,  2.12393562e+01,  1.73919306e+01,  4.53784911e+01,
        1.41643720e+01,  1.37905362e+01,  2.62910136e+01,  6.39612812e+01,
       -2.20000000e+04,  1.37905362e+01,  1.26560006e+01,  2.05652309e+01,
        1.77254850e+02,  7.65797569e+01,  4.50427632e+01,  3.43724390e+01,
        1.26560006e+01,  2.05652309e+01,  6.33170209e+01,  9.00972107e+01,
        6.06649260e+01,  3.64134071e+01,  1.76325032e+02,  1.10796437e+02,
        1.27379415e+02,  1.71522826e+02,  4.36932426e+01,  7.55931071e+01])

In [122]:
np.save('debiased_matrix_alt_orig_reshaped',debiased_matrix_alt_new)